As we have extracted pages from the survey report dumped in a pickle file before. Now, we will be using that pickle dumped file to read the data 

In [1]:
class Extracted_Page:
    def __init__(self, name):
        self.name = name
        self.text = ''
        self.tables = []
        self.Images = []

In [2]:
class Table_in_report:
    def __init__(self, name):
        self.name = name
        self.page_number = -1
        self.data_frame = []

In [164]:
import pickle
with open('extracted_Pages.pickle', 'rb') as handle:
    extracted_Pages = pickle.load(handle)

In [128]:
data_of_interest =['cp value','cp potential','cp stab potential','field gradient','anode consumption']


In [137]:
Important_Parameters = ['equipment description','location','zoneid','date','data analysis year','location comment','measurement comment','part number']

In [8]:
qualitative_comments = ['satisfactory','good' ]

In [20]:
extracted_Pages['page_18'].tables

[]

In [169]:
location_finding_strings = ['equipment description']
Location_available = []
for page in extracted_Pages:
    for table in extracted_Pages[page].tables:
        df = table.data_frame
        #adding all the terms to the list related to the location finding strings
        [Location_available.extend(df[name].tolist()) for name in df.columns for s in location_finding_strings if
         name.lower().startswith(s)]


In [55]:
#Location_available

In [170]:
extracted_Pages['page_147'].tables[0].data_frame

,Connector,Equipment Description,CP Value\n(mV)
0,54-1,Chain 1 Connector Tube,-957
1,54-2,Chain 2 Connector Tube,RA
2,54-3,Chain 3 Connector Tube,-957
3,54-4,Chain 4 Connector Tube,-869
4,54-4,Chain 4 Connector Tube,-870
5,54-5,Chain 5 Connector Tube,-663
6,54-6,Chain 6 Connector Tube,-643
7,54-7,Chain 7 Connector Tube,-659


In [180]:
tables_with_CP_readings = []
for page in extracted_Pages:
    for table in extracted_Pages[page].tables:
        for column in table.data_frame.columns:
            if any(column.lower().startswith(data) for data in data_of_interest):
                tables_with_CP_readings.append(table)
                continue

In [184]:
[print(df.name, df.data_frame.shape, df.page_number) for df in data_frames_with_CP_readings]

 south buoy steel cp readings  
 (4, 4) 41
 north ool cp reading  
 (1, 2) 94
 south ool cp reading 
 (1, 2) 100
 cp survey: upper chains  
 (7, 3) 117
No_name (8, 3) 147


[None, None, None, None, None]

In [173]:
#removing the  dataframes which are empty

for page in extracted_Pages:
    ini_tables_list = extracted_Pages[page].tables
    
    new_tables_list = ini_tables_list
    new_tables_list = [table for table in ini_tables_list if not table.data_frame.empty]
            
    if len(tables_list) != len(new_tables_list):
        new_page = Extracted_Page(page)
        new_page.text = extracted_Pages[page].text
        new_page.tables = new_tables_list
        
        extracted_Pages[page] = new_page
        

In [70]:
#[print(key, len(extracted_Pages[key].tables)) for key in extracted_Pages]

In [186]:
 extracted_Pages['page_147'].tables[0].data_frame.columns

Index(['Connector', 'Equipment Description', 'CP Value\n(mV)'], dtype='object')

In [210]:
df

,Connector,Equipment Description,CP Value\n(mV)
0,54-1,Chain 1 Connector Tube,-957
1,54-2,Chain 2 Connector Tube,RA
2,54-3,Chain 3 Connector Tube,-957
3,54-4,Chain 4 Connector Tube,-869
4,54-4,Chain 4 Connector Tube,-870
5,54-5,Chain 5 Connector Tube,-663
6,54-6,Chain 6 Connector Tube,-643
7,54-7,Chain 7 Connector Tube,-659


In [201]:
df = extracted_Pages['page_147'].tables[0].data_frame

In [217]:
#isinstance(df.iloc[0,2], str) 
for i in range(3):
    for j in range(4):
        try:
            print(float(df.iloc[j,i]))
        except ValueError:
            continue
        
            print('nnn')



-957.0
-957.0
-869.0


In [118]:
class Reading_value:
    def __init__(self, value):
        self.value = value
        self.location = ''
        self.unit = ''
        self.date = ''
        self.equipment_description = ''
    

In [218]:
unwanted_words_in_title = ['readings', 'cp', 'survey', ':']
readings_with_location = []
for CP_reading_table in tables_with_CP_readings:
    d_f = CP_reading_table.data_frame 
    
    print(d_f.shape)
    location_name = CP_reading_table.name
    for s in unwanted_words_in_title:
        location_name = location_name.replace(s,'')
    
    size_of_columns = len(d_f.columns)
    print(d_f.columns)
    for first_loop_col_idx, column in enumerate(d_f.columns):   
        if any(column.lower().startswith(data) for data in data_of_interest):
            print(column.lower())
            for i in range(len(d_f.index)):
                data_value = d_f.iloc[i, first_loop_col_idx]
                
                #if not isinstance(data_value, str) :
                try:
                    float(data_value)
                except ValueError:
                    continue

                new_value = Reading_value(data_value)
                new_value.unit = column
                print(data_value)
                    
                    #explore all the data in the same row
                for new_col_idx in range(size_of_columns):
                    if new_col_idx != first_loop_col_idx: #explore all except the previous one
                        secondary_data = d_f.iloc[i, new_col_idx]
                        if not isinstance(secondary_data, str) and d_f.columns[new_col_idx] == column:
                            continue
                            
                            #if any(st == d_f.columns[new_col_idx] for st in Important_parameters):
                        if d_f.columns[new_col_idx].lower() == 'location':
                            new_value.location = secondary_data
                            
                        if d_f.columns[new_col_idx].lower() == 'equipment description' :
                            new_value.equipment_description = secondary_data
                                
                readings_with_location.append(new_value)

(4, 4)
Index(['Task\nNumber', 'Equipment Description', 'Location', 'CP Value\n(mV)'], dtype='object')
cp value
(mv)
-1070
-1062
-1082
-1077
(1, 2)
Index(['Location', 'CP Value (mV)'], dtype='object')
cp value (mv)
-1084
(1, 2)
Index(['Location', 'CP Value (mV)'], dtype='object')
cp value (mv)
-1085
(7, 3)
Index(['Task', 'Equipment Description', 'CP Value\n(mV)'], dtype='object')
cp value
(mv)
-948
-948
-940
-865
-668
-664
-668
(8, 3)
Index(['Connector', 'Equipment Description', 'CP Value\n(mV)'], dtype='object')
cp value
(mv)
-957
-957
-869
-870
-663
-643
-659


In [219]:
[print(a.value,',', a.location,',', a.equipment_description) for a in readings_with_location]

-1070 , 0deg , Buoy Bottom Plating
-1062 , 180deg , Buoy Bottom Plating
-1082 , 0deg , Buoy Outer Shell Plating
-1077 , 180deg , Buoy Outer Shell Plating
-1084 , Flex-Joint Body , 
-1085 , Flex-Joint Body , 
-948 ,  , Mooring Chain 1 - Link 5
-948 ,  , Mooring Chain 2 - Link 5
-940 ,  , Mooring Chain 3 - Link 5
-865 ,  , Mooring Chain 4 - Link 5
-668 ,  , Mooring Chain 5 - Link 5
-664 ,  , Mooring Chain 6 - Link 5
-668 ,  , Mooring Chain 7 - Link 5
-957 ,  , Chain 1 Connector Tube
-957 ,  , Chain 3 Connector Tube
-869 ,  , Chain 4 Connector Tube
-870 ,  , Chain 4 Connector Tube
-663 ,  , Chain 5 Connector Tube
-643 ,  , Chain 6 Connector Tube
-659 ,  , Chain 7 Connector Tube


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]